In [1]:
# ======================================================
#    Blocks World Program
#
#      This program was introduced in Chapter 8.
#
#      CLIPS Version 6.0 Example
#
#      To execute, merely load, reset and run.
# ======================================================

In [2]:
from pyknow import *

In [3]:
class Goal(Fact):
    pass

class Stack(Fact):
    pass

In [4]:
class Blocks(KnowledgeEngine):
    @DefFacts()
    def initial_state(self):
        yield Stack("A", "B", "C")
        yield Stack("D", "E", "F")
        yield Goal(move="C", on_top_of="E")
        yield Stack()
    
    @Rule(
        AS.goal << Goal(move=MATCH.block1, on_top_of=MATCH.block2),
        AS.stack1 << Stack(MATCH.block1, +MATCH.rest1),
        AS.stack2 << Stack(MATCH.block2, +MATCH.rest2)
    )
    def move_directly(self, goal, block1, block2, stack1, stack2, rest1, rest2):
        self.retract(goal)
        self.retract(stack1)
        self.retract(stack2)
        self.declare(Stack(*rest1))
        self.declare(Stack(block1, block2, *rest2))
        print(block1, "moved on top of", block2)
        
    @Rule(
        AS.goal << Goal(move=MATCH.block1, on_top_of="floor"),
        AS.stack1 << Stack(MATCH.block1, +MATCH.rest)
    )
    def move_to_floor(self, goal, stack1, block1, rest):
        self.retract(goal)
        self.retract(stack1)
        self.declare(Stack(block1))
        self.declare(Stack(*rest))
        print(block1, "moved on top of floor")

    @Rule(
        Goal(move=MATCH.block1),
        Stack(MATCH.top, +MATCH.others),
        TEST(lambda block1, others: block1 in others)
    )
    def clear_upper_block(self, top):
        self.declare(Goal(move=top, on_top_of="floor"))
    
    @Rule(
        Goal(on_top_of=MATCH.block1),
        Stack(MATCH.top, +MATCH.others),
        TEST(lambda block1, others: block1 in others)
    )
    def clear_lower_block(self, top):
        self.declare(Goal(move=top, on_top_of="floor"))

In [5]:
ke=Blocks()
ke.reset()
ke.run()

D moved on top of floor
A moved on top of floor
B moved on top of floor
C moved on top of E
